# Noisy state preparation and measurement

The QLM's noise models include noisy state preparations and measurements. To describe those, you will need to specify the initial density matrix as well as the observable corresponding to the measurement.


### Simple example

Here is a code snippet where we specify a perfect initial state $|0\rangle \langle 0|$ and imperfect measurement:

In [ ]:
import numpy as np
from qat.hardware import DefaultGatesSpecification, HardwareModel

eps=0.1

gates_spec = DefaultGatesSpecification(gate_times = {"state_prep": 1, "meas": 1, "X": 10},
                                       state_prep=np.array([[1.0, 0.0],
                                                           [0.0, 0.0]]),
                                       meas=np.array([[eps, 0.0],
                                                      [0.0, 1.0-eps]]))

hw_model = HardwareModel(gates_spec)

Note that we suppose that the initial density matrix is factorized among the different qubits:

$$\rho_0 = \rho^{(1)} \otimes \rho^{(2)} \dots \otimes \rho^{(n)}$$

and similarly for the measurement observable.

You should input valid density matrices and observables in the fields ``state_prep`` and ``meas``.

### Qubit-wise specification

One can also specify different initial states for each qubit by replacing the ``np.array`` by a dictionary, with one key per qubit:


In [ ]:
eps0, eps1 = 0.1, 0.05
gates_spec = DefaultGatesSpecification(gate_times = {"state_prep": 1, "meas": 1, "X": 10},
                                       state_prep={0: np.array([[1.0, 0.0],
                                                           [0.0, 0.0]]),
                                                   1: np.array([[1.0, 0.0],
                                                           [0.0, 0.0]])},
                                       meas={0: np.array([[eps0, 0.0],
                                                         [0.0, 1.0-eps0]]),
                                             1: np.array([[eps1, 0.0],
                                                        [0.0, 1.0-eps1]])}
                                      )
hw_model = HardwareModel(gates_spec)

### Running a noisy computation

We now run a computation with a QPU affected by these SPAM errors:

In [ ]:
from qat.noisy import NoisyQProc

qpu_spam = NoisyQProc(hardware_model=hw_model)

In [ ]:
from qat.lang.AQASM import X, Program
prog = Program()
reg = prog.qalloc(2)
prog.apply(X, reg[0])
job = prog.to_circ().to_job()

results = qpu_spam.submit(job)
for sample in results:
    print(sample)